# 전처리한 데이터를 불러오자

In [ ]:
import pandas as pd
from glob import glob
import os
import numpy as np
from tqdm import tqdm, tqdm_notebook
import plotly.offline as pyo
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from datetime import datetime
from datetime import date
import time
from tqdm import tqdm

In [ ]:
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
pd.options.display.float_format = '{:.4f}'.format

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
%matplotlib inline
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import platform

path = "c:/Windows/Fonts/malgun.ttf"
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)  

In [ ]:
#pd.set_option('display.max_columns',None) # 전체 열 출력하기
#pd.set_option('display.max_rows',None) # 전체 행 출력하기

In [ ]:
# go to file
os.chdir('../DATA_raw2')

In [ ]:
user = pd.read_pickle("../TBWA/user_fin2.pkl")
order = pd.read_pickle("../TBWA/order_fin.pkl")

In [ ]:
print(len(user))
print(len(order))

1907989
4928381


# 1. 프로모션 진행 현황
- 총 진행 개수, 연도별, 월 별 각 진행 개수, 평균 행사 기간 파악

## 1.1 기간별 진행 현황

In [ ]:
promo = pd.read_excel('file/ABC 프로모션 list2.xlsx')

In [ ]:
promo

,년도,채널,행사명,행사 내용,행사 구분,시작일,종료일
0,2022,온라인,새해 행사,"호랑이띠 회원 22%할인쿠폰, 멤버십 10%할인쿠폰 증정",쿠폰,2022-01-01,2022-01-13
1,2022,온라인,신년 행사,신상품 10%할인쿠폰 증정 (온/오프라인 동시),쿠폰,2022-01-14,2022-01-24
2,2022,온라인,설 행사,최대 2만원 할인쿠폰 등 4종 쿠폰 증정,쿠폰,2022-01-25,2022-02-06
3,2022,온라인,발렌타인,"1족 구매 시 10%, 2족 이상 구매 시 20%할인 (온/오프라인 동시)","단순 할인, 조건 할인",2022-02-08,2022-02-14
4,2022,온라인,아트닷컴 2주년,"20%할인, 추가 10%할인쿠폰, 당일/새벽배송 무료 쿠폰 증정.\n나이키, 아디다...","쿠폰, 단순 할인",2022-02-18,2022-02-27
...,...,...,...,...,...,...,...
142,2019,오프라인,할로윈데이,키즈 상품 1족 구매 시 10% / 2족 이상 구매 시 20% 할인,"단순 할인, 조건 할인",2019-10-25,2019-10-31
143,2019,오프라인,빼빼로데이,1족 구매 시 10% / 2족 이상 구매 시 20% 할인,"단순 할인, 조건 할인",2019-11-07,2019-11-11
144,2019,오프라인,블랙 ABC데이,결산세일 최대 70%할인 (온/오프라인 동시),단순 할인,2019-11-21,2019-12-15
145,2019,오프라인,크리스마스,"신상품 20%할인,",단순 할인,2019-12-16,2019-12-25


In [ ]:
promo['c_시작연도'] = promo['시작일'].dt.year
promo['c_시작월'] = promo['시작일'].dt.month
promo['c_시작연도'] = promo['c_시작연도'].astype(str)
promo['c_시작월'] = promo['c_시작월'].astype(str)

In [ ]:
data = promo.groupby(['c_시작연도','채널'], as_index=False).agg({'행사명' : 'count'})
fig = px.bar(data, x='c_시작연도', y='행사명', color = '채널', color_discrete_map={
        '오프라인': '#CC3333','온라인': '#FFCC33'}, title = '연도별 행사개수', text_auto = '.2s' )
fig.update_layout(xaxis_title='행사 연도',
                  yaxis_title='개수')
fig.show()

In [ ]:
data = promo.groupby(['c_시작연도','채널'], as_index=False).agg({'행사명' : 'count'})
data

,c_시작연도,채널,행사명
0,2019,오프라인,17
1,2019,온라인,24
2,2020,오프라인,14
3,2020,온라인,21
4,2021,오프라인,14
5,2021,온라인,22
6,2022,오프라인,15
7,2022,온라인,20


- 프로모션 개수는 2020-2022년 동안에는 비슷하게 유지
- 오프라인 구매가 강세인 것에 비해 프로모션은 온라인에서 더 많음 (이유가 왤까)

In [ ]:
promo2 = promo.copy()
list = []
for i in range(len(promo2)) :
    if promo2['시작일'][i].month != promo2['종료일'][i].month :
        list.append([promo2['시작일'][i].month, promo2['종료일'][i].month])
    else : 
        list.append(promo2['시작일'][i].month)
promo2['demo'] = list
promo2 = promo2.join(promo2['demo'].apply(lambda x: pd.Series(x)).stack().reset_index(1,name='c_진행월').drop('level_1', axis=1))
promo2 = promo2.drop(labels = 'demo', axis=1)
promo2['c_진행월'] = promo2['c_진행월'].astype(int)

In [ ]:
data = promo2.groupby(['c_진행월','채널'], as_index=False).agg({'행사명' : 'count'})
fig = px.bar(data, x='c_진행월', y='행사명', color = '채널', title = '월별 행사건수 (연도 통합)', color_discrete_map={
        '오프라인': '#CC3333','온라인': '#FFCC33'},text_auto = '.2s' )
fig.update_layout(xaxis_title='행사 월',
                  yaxis_title='개수')

In [ ]:
data

,c_진행월,채널,행사명
0,1,오프라인,7
1,1,온라인,10
2,2,오프라인,11
3,2,온라인,14
4,3,오프라인,5
5,3,온라인,10
6,4,오프라인,4
7,4,온라인,10
8,5,오프라인,6
9,5,온라인,7


- 2월 > 12월 > 7월 순으로 많이 함
- 아래 연도별로 봤을 때도 겨울, 여름에 집중

In [ ]:
data = promo2.groupby(['c_시작연도', 'c_진행월','채널'], as_index=False).agg({'행사명' : 'count'})
for i in data['c_시작연도'].unique() : 
    fig = px.bar(data[data['c_시작연도']==i], x='c_진행월', y='행사명',color = '채널', title =  i+'년 월별 행사건수',color_discrete_map={
        '오프라인': '#CC3333','온라인': '#FFCC33'},text_auto = '.2s' )
    fig.update_layout(xaxis_title='행사 월',
                  yaxis_title='개수')
    
    fig.show()

## 1.2 행사 종류 구분
- 구분 방식 : 쿠폰, 단순 할인(=하나만 사도 할인), 조건 할인(ex. 2족 이상 구매시 or 키즈상품 구매 시, 15만원 구매 시), 사은품(=무조건 주는 거), 경품(=이벤트 추첨을 통해서 주는 거) -> 쿠폰, 단순할인, 조건할인으로 나눔, 사은품, 경품같은 경우에는 보조적인 프로모션으로 보임
- 중복 시 모두 반영

In [ ]:
promo2['demo'] = promo2['행사 구분'].str.replace(" ", "")
promo2['demo'] = promo2['demo'].str.split(',')
promo2 = promo2.join(promo2['demo'].apply(lambda x: pd.Series(x)).stack().reset_index(1,name='c_행사구분').drop('level_1', axis=1))
promo2 = promo2.drop(labels = 'demo', axis=1)

In [ ]:
data = promo2.groupby(['c_시작연도' ,'c_시작월','c_행사구분'], as_index=False).agg({'행사명': 'count'}).sort_values(by=['c_시작연도','c_시작월', '행사명'], ascending = [True,True, False])
data.columns = ['행사 연도', '행사 월', '행사 구분', '행사 개수']
data.groupby(['행사 월', '행사 구분']).agg({'행사 개수': 'mean'}).sort_values(by=['행사 월', '행사 개수'], ascending = [True, False])

행사 개수
행사 월 행사 구분       
1    단순할인  5.5000
     쿠폰    4.2500
     조건할인  3.0000
10   단순할인  5.0000
     조건할인  3.3333
     쿠폰    1.0000
11   단순할인  9.5000
     경품    4.0000
     쿠폰    3.0000
     조건할인  1.0000
12   단순할인  4.5000
     조건할인  2.0000
     쿠폰    1.0000
2    단순할인  4.7500
     사은품   4.0000
     경품    2.5000
     쿠폰    2.2500
     조건할인  1.5000
3    쿠폰    5.0000
     단순할인  4.0000
     조건할인  1.5000
4    조건할인  6.6667
     단순할인  6.5000
     경품    5.0000
     쿠폰    2.5000
5    쿠폰    4.5000
     단순할인  3.3333
     조건할인  2.0000
6    단순할인  6.2500
     경품    4.0000
     조건할인  1.0000
     쿠폰    1.0000
7    단순할인  8.7500
     경품    4.0000
     사은품   4.0000
8    단순할인  4.5000
     경품    4.0000
     쿠폰    3.3333
     사은품   1.0000
     조건할인  1.0000
9    단순할인  4.7500
     조건할인  4.5000
     쿠폰    3.5000

In [ ]:
data = promo2.groupby(['c_행사구분']).agg({'행사명': 'count'}).sort_values(by=['행사명'], ascending = False)
data.columns = ['행사 개수']
data

,행사 개수
c_행사구분,
단순할인,253
쿠폰,86
조건할인,79
경품,39
사은품,9


In [ ]:
promo2 = promo.copy()
promo2['c_시작월'] = promo2['c_시작월'].astype(int)
promo2['demo'] = promo2['행사 구분'].str.replace(" ", "")
promo2['demo'] = promo2['demo'].str.split(',')
promo2 = promo2.join(promo2['demo'].apply(lambda x: pd.Series(x)).stack().reset_index(1,name='c_행사구분').drop('level_1', axis=1))
promo2 = promo2.drop(labels = 'demo', axis=1)
data = promo2.groupby(['c_시작연도','c_행사구분']).agg({'행사명': 'count'}).sort_values(by=['c_시작연도', '행사명'], ascending = False)
data.columns = ['행사 개수']
data

행사 개수
c_시작연도 c_행사구분       
2022   단순할인       25
       조건할인       13
       쿠폰         11
       경품          5
       사은품         2
2021   단순할인       30
       조건할인       10
       쿠폰         10
       경품          6
2020   단순할인       27
       조건할인       10
       쿠폰          8
       경품          1
       사은품         1
2019   단순할인       24
       쿠폰         15
       조건할인       10

In [ ]:
# 행사 기간
promo['행사진행기간'] = promo['종료일'] - promo['시작일']
promo['행사진행기간'] = promo['행사진행기간'].dt.days + 1
df = promo.groupby('c_시작연도', as_index = False).agg({'행사진행기간' : 'mean'})
df

,c_시작연도,행사진행기간
0,2019,13.2195
1,2020,16.0571
2,2021,16.7500
3,2022,20.9714


In [ ]:
df = promo.groupby('채널', as_index = False).agg({'행사진행기간' : 'mean'})
df

,채널,행사진행기간
0,오프라인,19.0667
1,온라인,14.9080


In [ ]:
promo2 = promo.copy()
promo2['행사진행기간'] = promo2['종료일'] - promo2['시작일'] 
promo2['행사진행기간'] = promo2['행사진행기간'].dt.days + 1
promo2['c_시작월'] = promo2['c_시작월'].astype(int)
promo2['demo'] = promo2['행사 구분'].str.replace(" ", "")
promo2['demo'] = promo2['demo'].str.split(',')
promo2 = promo2.join(promo2['demo'].apply(lambda x: pd.Series(x)).stack().reset_index(1,name='c_행사구분').drop('level_1', axis=1))
promo2 = promo2.drop(labels = 'demo', axis=1)
promo2.groupby('c_행사구분', as_index = False).agg({'행사진행기간' : 'mean'}).sort_values(by=['행사진행기간'], ascending = False).reset_index(drop=True)

,c_행사구분,행사진행기간
0,사은품,22.0000
1,경품,19.3333
2,단순할인,17.8019
3,쿠폰,15.1591
4,조건할인,13.0233


## 1.3 행사 주기
- 전체적인 행사 주기, 행사별 행사주기, 채널 별 행사주기

In [ ]:
#전체적인 행사 주기
promo3 = promo.copy()
promo3 = promo3.sort_values(by=['채널', '시작일'], ascending = True).reset_index(drop=True)
promo3['직전시작일'] = promo3.groupby(['채널']).시작일.shift()
promo3['행사주기'] = promo3['시작일'] - promo3['직전시작일']
promo3['행사주기'] = promo3['행사주기'].dt.days
promo3['행사주기'] = np.where(promo3['행사주기'].isna(), 0, promo3['행사주기'])
promo3['행사주기'].mean() #19일에 한번

19.523809523809526

In [ ]:
# 채널 별 행사주기
promo3.groupby(['채널'], as_index=False).agg({'행사주기':'mean'}).sort_values(by=['행사주기'], ascending = True).reset_index(drop=True)

,채널,행사주기
0,온라인,16.6552
1,오프라인,23.6833


In [ ]:
#행사구분 별 행사주기
promo3['demo'] = promo3['행사 구분'].str.replace(" ", "")
promo3['demo'] = promo3['demo'].str.split(',')
promo3 = promo3.join(promo3['demo'].apply(lambda x: pd.Series(x)).stack().reset_index(1,name='c_행사구분').drop('level_1', axis=1))
promo3 = promo3.drop(labels = 'demo', axis=1)
promo3.groupby('c_행사구분', as_index = False).agg({'행사주기' : 'mean'}).sort_values(by=['행사주기'], ascending = True).reset_index(drop=True)

,c_행사구분,행사주기
0,경품,13.0833
1,쿠폰,14.7500
2,사은품,19.6667
3,단순할인,20.8774
4,조건할인,21.9070


# 2. 프로모션 별 이용률
- 온라인 / 오프라인 이용률
- 종류 구분




- user 중에 최초가입일 1900년인 사람 제거
- user 중에 최종가입 < 최종탈퇴일인 사람 제거 -> 구매 240건 제거. 
- 프모 중 해당 기간에 산 사람

In [ ]:
promo = promo.sort_values(by=['시작일'], ascending = True).reset_index(drop=True)
promo = promo.reset_index()

In [ ]:
# 온라인 구매 / 오프라인 구매 나눠서

In [ ]:
promo_online = promo[promo['채널'] == '온라인'].reset_index(drop=True)
promo_offline = promo[promo['채널'] == '오프라인'].reset_index(drop=True)
print(len(promo_online)) #온라인 프로모션 개수
print(len(promo_offline)) #오프라인 프로모션 개수

87
60


In [ ]:
#order_online = order[(order['구매처'] == 'ABC Mobile') | (order['구매처'] == 'ABC APP') | 
#              (order['구매처'] == 'ABC PC')].reset_index(drop=True)
#order_offline = order[(order['구매처'] != 'ABC Mobile') & (order['구매처'] != 'ABC APP') &
#              (order['구매처'] != 'ABC PC')].reset_index(drop=True)
#print(len(order))
#print(len(order_online))
#print(len(order_offline))

In [ ]:
#a = []
#for i in tqdm(range(len(order_online))) : 
#    list = []
#    for j in range(len(promo_online)) :
#        if (order_online.구매일자[i] >= promo_online.시작일[j]) & (order_online.구매일자[i] <= promo_online.종료일[j]):
#            list.append(promo_online['index'][j]) 
#        else :
#            pass
#    a.append(list)
#order_online['행사참여'] = a
#order_online.to_pickle("../TBWA/order_online.pkl")

In [ ]:
#b = []
#for i in tqdm(range(len(order_offline))) : 
#    list = []
#    for j in range(len(promo_offline)) :
#        if (order_offline.구매일자[i] >= promo_offline.시작일[j]) & (order_offline.구매일자[i] <= promo_offline.종료일[j]):
#            list.append(promo_offline['index'][j]) 
#        else :
#            pass
#    b.append(list)
#order_offline['행사참여'] = b
#order_offline.to_pickle("../TBWA/order_offline.pkl")

In [ ]:
order_online = pd.read_pickle("../TBWA/order_online.pkl")
order_offline = pd.read_pickle("../TBWA/order_offline.pkl")

In [ ]:
order_com = pd.concat([order_online, order_offline], axis=0)
order_com = order_com[order_com.고유번호.isin(order.고유번호.unique())]

## 2.1 행사기간에 구매한 건 vs 행사기간 아닐 때 구매한 건

In [ ]:
# 거래건수 중에서 행사기간에 구매한 건 vs 행사기간 아닐 때 구매한 건
list1 = ['기간 내 구매', '기간 외 구매']
list2 = [3915863, 1058344]
df = pd.DataFrame({'기간' : list1, '구매 수':list2})
df

In [ ]:
fig = px.pie(df, values='구매 수', names='기간', title='프로모션 기간 내 구매한 건수는 전체의 78%', color = '기간', 
             color_discrete_map={ '기간 내 구매': '#CD5C5C','기간 외 구매': '#4169E1'}) 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show('notebook')

In [ ]:
demo = order_com[['고유번호', '구매일자','결제가','구매처','행사참여']]
demo.행사참여 = demo.행사참여.astype(object)
demo2 = demo.explode('행사참여')
demo2['행사참여'] = demo2['행사참여'].fillna(value='행사x')
demo2 = demo2.rename(columns = {'행사참여' : '행사번호'}).reset_index(drop=True)
demo2
# 회원 중에서 행사기간에 회원 수 vs 행사기간 아닐 때 구매한 회원 수ㅜ
list1 = ['기간 내 이용한 회원', '이용하지 않은 회원']
list2 = [len(user[user.고유번호.isin(demo2.고유번호.unique())]) , len(user[~user.고유번호.isin(demo2.고유번호.unique())])]
df = pd.DataFrame({'기간' : list1, '회원 수':list2})
df
fig = px.pie(df, values='회원 수', names='기간', title='프로모션 이용률은 전체 회원수의 71.6%', color = '기간', 
             color_discrete_map={ '기간 내 이용한 회원': '#CD5C5C','이용하지 않은 회원': '#4169E1'}) 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show('notebook')

In [ ]:
demo3 = demo2[demo2['행사번호'] != '행사x']
aa = demo3.groupby(['고유번호'], as_index = False).agg({'행사번호' : 'nunique'})
list = []
for i in range(len(aa)) : 
    if aa['행사번호'][i] == 1:
        list.append('1회 참여')
    elif aa['행사번호'][i] == 2:
        list.append('2회 참여')
    elif aa['행사번호'][i] == 3:
        list.append('3회 참여')
    elif aa['행사번호'][i] == 4:
        list.append('4회 참여')
    elif aa['행사번호'][i] == 5:
        list.append('5회 참여')
    else:
        list.append('6회 이상 구매')
aa['참여횟수'] = list
one = aa.groupby(['참여횟수'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['고유번호'], ascending = True).reset_index(drop=True)
fig = px.pie(one, values='고유번호', names='참여횟수', title='프로모션 참여한 고객이 또 참여하는지, 1회만 참여하는지', color = '참여횟수') 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
one['ratio'] = one.고유번호 / one.고유번호.sum() * 100
one

In [ ]:
# 3회 이상인 회원
aa = user[user.고유번호.isin(aa[(aa['행사번호'] != 1) & (aa['행사번호'] != 2)].고유번호)]

## 2.2 147개 각 프로모션 별 이용률

- 진행기간. 이 다름을 고려하여 '1일' 기준으로 
- 프로모션 별 이용고객 수 : 동일 고객이면 한번만 카운트. 고유번호 nunique
- 프로모션 별 구매 건수 : 고유번호 count
- 프로모션 별 인당 평균 구매 건수 : 
- 프로모션 별 평균 구매 금액

In [ ]:
demo = order_com[['고유번호', '구매일자','결제가','행사참여']]
demo.행사참여 = demo.행사참여.astype(object)
demo2 = demo.explode('행사참여')
demo2['행사참여'] = demo2['행사참여'].fillna(value='행사x')

In [ ]:
# 이용고객수
df1 = demo2.groupby(['행사참여'], as_index=False).agg({'고유번호':'nunique'})
df1.columns = ['행사번호', '이용고객수']
df1 = df1[:147]

In [ ]:
# 구매건수
df2 = demo2.groupby(['행사참여'], as_index=False).agg({'고유번호':'count'})
df2.columns = ['행사번호', '구매건수']
df2 = df2[:147]

In [ ]:
# 인당 평균 구매 건수
df3 = pd.merge(df2, df1, on = '행사번호', how = 'inner')
df3['1인 평균 구매건수'] = df3['구매건수'] / df3['이용고객수']

In [ ]:
# 구매금액 합산
df4 = demo2.groupby(['행사참여'], as_index=False).agg({'결제가':'sum'})
df4 = df4[:147]
df4.columns = ['행사번호', '총구매금액']
df4 = pd.merge(df3, df4, on='행사번호', how = 'inner')
df4['1인 평균 구매금액'] = df4['총구매금액'] / df4['이용고객수']

In [ ]:
promo = promo.rename(columns = {'index' : '행사번호'})

In [ ]:
promo_com = pd.merge(promo, df4, on='행사번호', how='inner')

In [ ]:
promo_com['1일 평균 구매건수'] = promo_com['구매건수'] / promo_com['행사진행기간']
promo_com['1일 평균 구매금액'] = promo_com['총구매금액'] / promo_com['행사진행기간']
promo_com['1일 이용고객수'] = promo_com['이용고객수'] / promo_com['행사진행기간']

In [ ]:
promo_com = promo_com[['채널', '행사명', '행사 구분', '시작일', '종료일','c_시작연도','c_시작월',  '행사진행기간', 
                       '이용고객수', '1일 이용고객수', '구매건수', '1인 평균 구매건수', '1일 평균 구매건수', '총구매금액','1인 평균 구매금액', '1일 평균 구매금액' ]]
promo_com.head() 

,채널,행사명,행사 구분,시작일,종료일,c_시작연도,c_시작월,행사진행기간,이용고객수,1일 이용고객수,구매건수,1인 평균 구매건수,1일 평균 구매건수,총구매금액,1인 평균 구매금액,1일 평균 구매금액
0,온라인,회원 혜택,쿠폰,2019-01-07,2019-01-13,2019,1,7,3733,533.2857,5128,1.3737,732.5714,228999900,61344.7361,32714271.4286
1,온라인,새학기 행사,쿠폰,2019-01-25,2019-02-28,2019,1,35,21176,605.0286,30643,1.4471,875.5143,1515887400,71585.1624,43311068.5714
2,오프라인,새학기 행사,단순 할인,2019-01-25,2019-02-12,2019,1,19,10723,564.3684,19267,1.7968,1014.0526,1104751500,103026.3452,58144815.7895
3,온라인,설 행사,쿠폰,2019-01-31,2019-02-10,2019,1,11,7180,652.7273,9875,1.3753,897.7273,495993500,69079.8747,45090318.1818
4,오프라인,설 행사,"단순 할인, 조건 할인",2019-01-31,2019-02-10,2019,1,11,7492,681.0909,13682,1.8262,1243.8182,778775100,103947.5574,70797736.3636


In [ ]:
pd.set_option('display.max_columns',None) # 전체 열 출력하기
pd.set_option('display.max_rows',None) # 전체 행 출력하기

In [ ]:
promo_com

In [ ]:
pd.set_option('display.max_columns',50) # 전체 열 출력하기
pd.set_option('display.max_rows',50) # 전체 행 출력하기

## 2.3 온라인 / 오프라인 이용률

In [ ]:
for i in [ '이용고객수','1일 이용고객수', '구매건수', '1일 평균 구매건수', '총구매금액', '1일 평균 구매금액']:
    event = promo_com.groupby(['채널'],as_index=False).agg({i : 'mean'})
    display(event)

- 행사진행기간이 오프라인이 더 긺
- 이를 감안하여 1일당 지표를 확인 -> 온라인에서 이용고객, 구매건수 많음

## 2.4 프로모션 구분 별 이용률

In [ ]:
demo = promo_com.copy()
demo['행사 구분'] = demo['행사 구분'].str.replace(" ", "")
demo['행사 구분'] = demo['행사 구분'].str.split(',')
promo_com2 = demo.join(demo['행사 구분'].apply(lambda x: pd.Series(x)).stack().reset_index(1,name='c_행사구분').drop('level_1', axis=1))
promo_com2 = promo_com2.drop(labels = '행사 구분', axis=1)
promo_com2.groupby(['c_시작연도','c_행사구분'])['이용고객수','1일 이용고객수', '구매건수', '1인 평균 구매건수', '1일 평균 구매건수', '총구매금액', 
'1인 평균 구매금액','1일 평균 구매금액'].mean()

#### 연도별 흐름 -온라인/오프라인 나눠서

In [ ]:
demo = promo_com.copy()
#demo = demo[demo['채널'] == '온라인']
demo['시작일'] = demo['시작일'].astype(str)
demo['시작연월'] = demo['시작일'].str[:7]
log = demo.groupby(['시작연월','채널'],as_index = False)['1일 이용고객수', '1일 평균 구매건수', '1일 평균 구매금액'].mean()
log['연도'] = log['시작연월'].str[:4].astype(int)
log['월'] = log['시작연월'].str[5:].astype(int)

In [ ]:
for i in ['1일 이용고객수', '1일 평균 구매건수', '1일 평균 구매금액'] :
    fig = px.line(log, x= '시작연월', 
              y= i, color = '채널',
              title= i + ' - 프로모션 이용 연도별 흐름')
    fig.update_layout(xaxis_title='연월',
                  yaxis_title='값')
    fig.show()

#### 월 별 패턴찾기

In [ ]:
demo = promo_com.copy()
demo = demo[demo['채널'] == '온라인']
demo['시작일'] = demo['시작일'].astype(str)
demo['시작연월'] = demo['시작일'].str[:7]
log = demo.groupby(['시작연월'],as_index = False)['1일 이용고객수', '1일 평균 구매건수', '1일 평균 구매금액'].mean()
log['연도'] = log['시작연월'].str[:4].astype(int)
log['월'] = log['시작연월'].str[5:].astype(int)

In [ ]:
for i in ['1일 이용고객수', '1일 평균 구매건수', '1일 평균 구매금액'] :
    fig = px.line(log, x= '월', 
              y= i,
              title= i + ' - 온라인 프로모션 이용 연도별 흐름', color = '연도')
    fig.update_layout(xaxis_title='월',
                  yaxis_title='값')
    fig.show()

In [ ]:
demo = promo_com.copy()
demo = demo[demo['채널'] == '오프라인']
demo['시작일'] = demo['시작일'].astype(str)
demo['시작연월'] = demo['시작일'].str[:7]
log = demo.groupby(['시작연월'],as_index = False)['1일 이용고객수', '1일 평균 구매건수', '1일 평균 구매금액'].mean()
log['연도'] = log['시작연월'].str[:4].astype(int)
log['월'] = log['시작연월'].str[5:].astype(int)

In [ ]:
for i in ['1일 이용고객수', '1일 평균 구매건수', '1일 평균 구매금액'] :
    fig = px.line(log, x= '월', 
              y= i,
              title= i + ' - 오프라인 프로모션 이용 연도별 흐름', color = '연도')
    fig.update_layout(xaxis_title='월',
                  yaxis_title='값')
    fig.show()

# 3. 프로모션 별 이용 회원 집계

## 3.1 프로모션 구분 별 이용 현황

In [ ]:
demo = promo_com.copy()
demo['행사 구분'] = demo['행사 구분'].str.replace(" ", "")
demo['행사 구분'] = demo['행사 구분'].str.split(',')
promo_com2 = demo.join(demo['행사 구분'].apply(lambda x: pd.Series(x)).stack().reset_index(1,name='c_행사구분').drop('level_1', axis=1))
promo_com2 = promo_com2.drop(labels = '행사 구분', axis=1)
aa = promo_com2.groupby(['c_시작연도','c_행사구분'])['이용고객수','1일 이용고객수', '구매건수', '1인 평균 구매건수', '1일 평균 구매건수', '총구매금액', 
'1인 평균 구매금액','1일 평균 구매금액'].mean()

In [ ]:
aa

In [ ]:
aa= promo_com2.groupby(['c_시작연도','c_행사구분'], as_index=False)['이용고객수','1일 이용고객수', '구매건수', '1인 평균 구매건수', '1일 평균 구매건수', '총구매금액', 
'1인 평균 구매금액','1일 평균 구매금액'].mean()
aa = aa[aa['c_시작연도'] == '2022']

In [ ]:
aa

## 3.2 2022년에 진행된 프로모션 종류 기준, 회원 특징

In [ ]:
demo = order_com[['고유번호', '구매일자','결제가','구매처','행사참여']]
demo.행사참여 = demo.행사참여.astype(object)
demo2 = demo.explode('행사참여')
demo2['행사참여'] = demo2['행사참여'].fillna(value='행사x')
demo2 = demo2.rename(columns = {'행사참여' : '행사번호'}).reset_index(drop=True)

In [ ]:
promo_sample = promo[['행사번호', '행사 구분']]
promo_sample['행사 구분'] = promo_sample['행사 구분'].str.replace(" ", "")
promo_sample['행사 구분'] = promo_sample['행사 구분'].str.split(',')
promo_sample = promo_sample.join(promo_sample['행사 구분'].apply(lambda x: pd.Series(x)).stack().reset_index(1,name='c_행사구분').drop('level_1', axis=1))
promo_sample = promo_sample.drop(labels = '행사 구분', axis=1)

In [ ]:
df = pd.merge(demo2, promo_sample, on = '행사번호', how= 'outer')
df['구매연도'] = df['구매일자'].dt.year
df = df[df['구매연도'] == 2022].reset_index(drop=True)

In [ ]:
df["지점키워드"] = '1'
kk = df[df["구매처"].str.contains("GS")]
kk["지점키워드"] = "GS"
df.loc[kk.index,:] = kk

kk = df[df["구매처"].str.contains("MS")]
kk["지점키워드"] = "MS"
df.loc[kk.index,:] = kk

kk = df[df["구매처"].str.contains("ST")]
kk["지점키워드"] = "ST"
df.loc[kk.index,:] = kk

kk = df[df["구매처"].str.contains("SP")]
kk["지점키워드"] = "SP"
df.loc[kk.index,:] = kk

kk = df[df["구매처"].str.contains("KM")]
kk["지점키워드"] = "KM"
df.loc[kk.index,:] = kk


kk = df[df["구매처"].str.contains("OTS")]
kk["지점키워드"] = "OTS"
df.loc[kk.index,:] = kk


kk = df[df["구매처"].str.contains("FS")]
kk["지점키워드"] = "FS"
df.loc[kk.index,:] = kk

kk = df[df["구매처"].str.contains("ES")]
kk["지점키워드"] = "ES"
df.loc[kk.index,:] = kk

kk = df[df["구매처"].str.contains("DS")]
kk["지점키워드"] = "DS"
df.loc[kk.index,:] = kk

kk = df[df["구매처"].str.contains("APP")]
kk["지점키워드"] = "APP"
df.loc[kk.index,:] = kk

kk = df[df["구매처"].str.contains("Mobile")]
kk["지점키워드"] = "Mobile"
df.loc[kk.index,:] = kk

kk = df[df["구매처"].str.contains("PC")]
kk["지점키워드"] = "PC"
df.loc[kk.index,:] = kk

kk = df[df["구매처"].str.contains("본사특판")]
kk["지점키워드"] = "본사특판"
df.loc[kk.index,:] = kk

In [ ]:
def place(x):
    if x=='APP':
        return 'APP'
    elif x== 'PC':
        return 'PC'
    elif x== 'Mobile':
        return 'Mobile'
    else :
        return '오프라인'

df['지점키워드2'] = df['지점키워드'].apply(place) 

- 프로모션 참여 o -> 포로모션 별 구매건수에서 차지하는 비중

In [ ]:
one = df.groupby(['c_행사구분'],as_index = False).agg({'고유번호' : 'count'}).sort_values(by=['고유번호'], ascending = True).reset_index(drop=True)
fig = px.pie(one, values='고유번호', names='c_행사구분', title='전체 건수에서 차지하는 비율', color = 'c_행사구분') 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
one['ratio'] = one.고유번호 / one.고유번호.sum()
one

,c_행사구분,고유번호,ratio
0,사은품,177902,0.0451
1,경품,371803,0.0942
2,쿠폰,734813,0.1863
3,조건할인,892471,0.2262
4,단순할인,1768029,0.4482


In [ ]:
df1 = df[df['c_행사구분']=='단순할인']
df1 = user[user.고유번호.isin(df1.고유번호.unique())] #79만명
df2 = df[df['c_행사구분']=='조건할인']
df2 = user[user.고유번호.isin(df2.고유번호.unique())] #46만명
df3 = df[df['c_행사구분']=='쿠폰']
df3 = user[user.고유번호.isin(df3.고유번호.unique())] #40만명
df4 = df[df['c_행사구분']=='경품']
df4 = user[user.고유번호.isin(df4.고유번호.unique())] #23만명
df5 = df[df['c_행사구분']=='사은품']
df5 = user[user.고유번호.isin(df5.고유번호.unique())] #11만명

## 3.3 2022 단순할인 회원 특징 
- 프로모션 기간 내 구매한 경험이 없는 고객이 전체의 19.4%
- 기간 내 구매한 경험이 있는 고객 중 할인기간에 구매한 회원의 특징

In [ ]:
yesuser = user[(user.고유번호.isin(df1.고유번호.unique())) | (user.고유번호.isin(df2.고유번호.unique()))|
(user.고유번호.isin(df3.고유번호.unique())) &(user.고유번호.isin(df4.고유번호.unique())) |
(user.고유번호.isin(df5.고유번호.unique())) ]

In [ ]:
# 2022년 프로모션 기간 내 구매 o vs x 구매 x
list1 = ['프로모션 기간 내 구매', '별개 구매']
list2 = [853990, 205895]
data = pd.DataFrame({'프로모션' : list1, '회원 수':list2})
fig = px.pie(data, values='회원 수', names='프로모션', title='프로모션 행사 사용 안하는 고객이 일단 19.4%', color = '프로모션', 
             color_discrete_map={ '프로모션 기간 내 구매': '#CD5C5C','별개 구매': '#4169E1'}) 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show('notebook')

In [ ]:
# 2022년 프로모션 기간 내 구매 o vs x 구매 x
list1 = ['단순 할인 이용 o 고객', '이용 x 고객']
list2 = [df1.고유번호.nunique(), df.고유번호.nunique()-df1.고유번호.nunique()]
data = pd.DataFrame({'프로모션' : list1, '회원 수':list2})
fig = px.pie(data, values='회원 수', names='프로모션', title='단순할인 이용 고객 75%', color = '프로모션', 
             color_discrete_map={ '단순 할인 이용 o 고객': '#CD5C5C','이용 x 고객': '#4169E1'}) 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show('notebook')

In [ ]:
# 성별
sex = df1.groupby(['성별'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['성별'], ascending = True)
sex['퍼센트'] = sex['고유번호'] / sex['고유번호'].sum() *100
sex

In [ ]:
# 연령별
age = df1.groupby(['연령대'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['연령대'], ascending = True)
age['퍼센트'] = age['고유번호'] / age['고유번호'].sum() *100
age

In [ ]:
age['연령대2']='60세 이상'
age['연령대2'] = np.where(age['연령대']=='19세 미만', '10대 이하',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='20-24세')|(age['연령대']=='25-29세'), '20대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='30~34세')|(age['연령대']=='35~39세'), '30대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='40~44세')|(age['연령대']=='45~49세'), '40대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='50~54세')|(age['연령대']=='55~59세'), '50대',age['연령대2'])

age = df1.groupby(['연령대2'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['연령대2'], ascending = True)
age['퍼센트'] = age['고유번호'] / age['고유번호'].sum() *100
age

In [ ]:
# 구매처 차이
df1 = df[df['c_행사구분']=='단순할인']

place = df1.groupby(['지점키워드2'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['고유번호'], ascending = True).reset_index(drop=True)
place['ratio'] = place.고유번호 / place.고유번호.sum() * 100
place

## 3.4 2022 조건할인 회원 특징 


In [ ]:
# 2022년 프로모션 기간 내 구매 o vs x 구매 x
list1 = ['조건할인 이용 o 고객', '이용 x 고객']
list2 = [df2.고유번호.nunique(), df.고유번호.nunique()-df2.고유번호.nunique()]
data = pd.DataFrame({'프로모션' : list1, '회원 수':list2})
fig = px.pie(data, values='회원 수', names='프로모션', title='조건할인 이용 고객 44.3%%', color = '프로모션', 
             color_discrete_map={ '조건할인 이용 o 고객': '#CD5C5C','이용 x 고객': '#4169E1'}) 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show('notebook')

- 전제조건이 붙으면 이용을 덜하게 되는 경향

In [ ]:
# 성별
sex = df2.groupby(['성별'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['성별'], ascending = True)
sex['퍼센트'] = sex['고유번호'] / sex['고유번호'].sum() *100
fig = px.pie(sex, values='고유번호', names='성별', title='성별 - 여성 > 남성', color = '성별', 
             color_discrete_map={ 'F': '#CD5C5C','M': '#4169E1'}) 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
# 연령별
age = df2.groupby(['연령대'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['연령대'], ascending = True)
age['퍼센트'] = age['고유번호'] / age['고유번호'].sum() *100
age

In [ ]:
age['연령대2']='60세 이상'
age['연령대2'] = np.where(age['연령대']=='19세 미만', '10대 이하',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='20-24세')|(age['연령대']=='25-29세'), '20대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='30~34세')|(age['연령대']=='35~39세'), '30대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='40~44세')|(age['연령대']=='45~49세'), '40대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='50~54세')|(age['연령대']=='55~59세'), '50대',age['연령대2'])
age = df2.groupby(['연령대2'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['연령대2'], ascending = True)
age['퍼센트'] = age['고유번호'] / age['고유번호'].sum() *100
age

In [ ]:
# 구매처 차이
df2 = df[df['c_행사구분']=='조건할인']

place = df2.groupby(['지점키워드2'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['고유번호'], ascending = True).reset_index(drop=True)
place['ratio'] = place.고유번호 / place.고유번호.sum() * 100
place

,지점키워드2,고유번호,ratio
0,PC,44407,4.9757
1,Mobile,103095,11.5516
2,APP,160266,17.9576
3,오프라인,584703,65.5151


## 3.3 2022 쿠폰 회원 특징 


In [ ]:
# 2022년 프로모션 기간 내 구매 o vs x 구매 x
list1 = ['쿠폰 이용 o 고객', '이용 x 고객']
list2 = [df3.고유번호.nunique(), df.고유번호.nunique()-df3.고유번호.nunique()]
data = pd.DataFrame({'프로모션' : list1, '회원 수':list2})
fig = px.pie(data, values='회원 수', names='프로모션', title='쿠폰 이용 고객 62.1%', color = '프로모션', 
             color_discrete_map={ '쿠폰 이용 o 고객': '#CD5C5C','이용 x 고객': '#4169E1'}) 
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show('notebook')

In [ ]:
# 성별
sex = df3.groupby(['성별'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['성별'], ascending = True)
sex['퍼센트'] = sex['고유번호'] / sex['고유번호'].sum() *100
sex

In [ ]:
# 연령별
age = df3.groupby(['연령대'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['연령대'], ascending = True)
age['퍼센트'] = age['고유번호'] / age['고유번호'].sum() *100
age

In [ ]:
age['연령대2']='60세 이상'
age['연령대2'] = np.where(age['연령대']=='19세 미만', '10대 이하',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='20-24세')|(age['연령대']=='25-29세'), '20대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='30~34세')|(age['연령대']=='35~39세'), '30대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='40~44세')|(age['연령대']=='45~49세'), '40대',age['연령대2'])
age['연령대2'] = np.where((age['연령대']=='50~54세')|(age['연령대']=='55~59세'), '50대',age['연령대2'])

age = df3.groupby(['연령대2'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['연령대2'], ascending = True)
age['퍼센트'] = age['고유번호'] / age['고유번호'].sum() *100
age

In [ ]:
# 구매처 차이
df3 = df[df['c_행사구분']=='쿠폰']

place = df3.groupby(['지점키워드2'],as_index = False).agg({'고유번호':'count'}).sort_values(by=['고유번호'], ascending = True).reset_index(drop=True)
place['ratio'] = place.고유번호 / place.고유번호.sum() * 100
place

## 3.4 2022 온라인 프로모션 이용고객 중 pc,모바일,앱으로 구매한 고객 비중

In [ ]:
demo = order_online[['고유번호', '구매일자','결제가','구매처','행사참여']]
demo.행사참여 = demo.행사참여.astype(object)
demo2 = demo.explode('행사참여')
demo2['행사참여'] = demo2['행사참여'].fillna(value='행사x')
demo2 = demo2.rename(columns = {'행사참여' : '행사번호'}).reset_index(drop=True)

In [ ]:
dd = demo2[demo2['행사번호'] != '행사x']
dd = pd.merge(dd, promo[['행사번호', '채널']], on = '행사번호', how = 'left')

In [ ]:
dd["지점키워드"] = '1'
kk = dd[dd["구매처"].str.contains("GS")]
kk["지점키워드"] = "GS"
dd.loc[kk.index,:] = kk

kk = dd[dd["구매처"].str.contains("MS")]
kk["지점키워드"] = "MS"
dd.loc[kk.index,:] = kk

kk = dd[dd["구매처"].str.contains("ST")]
kk["지점키워드"] = "ST"
dd.loc[kk.index,:] = kk

kk = dd[dd["구매처"].str.contains("SP")]
kk["지점키워드"] = "SP"
dd.loc[kk.index,:] = kk

kk = dd[dd["구매처"].str.contains("KM")]
kk["지점키워드"] = "KM"
dd.loc[kk.index,:] = kk


kk = dd[dd["구매처"].str.contains("OTS")]
kk["지점키워드"] = "OTS"
dd.loc[kk.index,:] = kk


kk = dd[dd["구매처"].str.contains("FS")]
kk["지점키워드"] = "FS"
dd.loc[kk.index,:] = kk

kk = dd[dd["구매처"].str.contains("ES")]
kk["지점키워드"] = "ES"
dd.loc[kk.index,:] = kk

kk = dd[dd["구매처"].str.contains("DS")]
kk["지점키워드"] = "DS"
dd.loc[kk.index,:] = kk

kk = dd[dd["구매처"].str.contains("APP")]
kk["지점키워드"] = "APP"
dd.loc[kk.index,:] = kk

kk = dd[dd["구매처"].str.contains("Mobile")]
kk["지점키워드"] = "Mobile"
dd.loc[kk.index,:] = kk

kk = dd[dd["구매처"].str.contains("PC")]
kk["지점키워드"] = "PC"
dd.loc[kk.index,:] = kk

kk = dd[dd["구매처"].str.contains("본사특판")]
kk["지점키워드"] = "본사특판"
dd.loc[kk.index,:] = kk

In [ ]:
def place(x):
    if x=='APP':
        return 'APP'
    elif x== 'PC':
        return 'PC'
    elif x== 'Mobile':
        return 'Mobile'
    else :
        return '오프라인'

dd['지점키워드2'] = dd['지점키워드'].apply(place) 

In [ ]:
# 주 구매처
place = dd.groupby('지점키워드2', as_index = False).agg({'고유번호':'count'})
place['ratio'] = place.고유번호 / place.고유번호.sum() * 100
place